In [1]:
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy


2018-01-12 22:07:59,295 : INFO : 'pattern' package not found; tag filters are not available for English


In [4]:
# get stopwords as set
# get Vocabulary and remove words with length less than 3 
# vocab is sorted in descending order 

en_stop = set(get_stop_words('en'))
vocab  = open("wordsEn.txt").read().splitlines()
vocab = [i for i in vocab  if len(i)>2]
vocab.sort(key=len, reverse=True)

vocab_set = set(vocab)


In [5]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))


# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    lemmatizer = WordNetLemmatizer()
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        
        # tokenize that string
        tokens =  word_tokenize(string)
        
        # remove stop words tokens and length two tokens
        tokens = [lemmatizer.lemmatize(i) for i in tokens if i not in en_stop and len(i)>2]

        #seperate words stuck together
        for i,token in enumerate(tokens):
            if token not in vocab_set:
                # both prefix and suffix incase one of the word is of len(2)
                # prefix match
                flag = 0
                for j in range(len(token)-2,2,-1):
                    if token[:j] in vocab_set:
                            tokens.append(token[:j])
                            tokens.append(token[j:])
                            del tokens[i]
                            flag = 1
                            break               
                
                if flag == 1:
                    continue 
                # suffix match
                flag = 0
                for j in range(1,len(token)-2):
                    if token[j:] in vocab_set:
                            tokens.append(token[:j])
                            tokens.append(token[j:])
                            flag =1
                            del tokens[i]                            
                            break               

        # again remove stop words and length two tokens AND WORDS not in vocab
        tokens = [i for i in tokens if i not in en_stop and len(i)>2 and i in vocab_set]                
        if len(tokens) == 0:
            print "empty document"
        yield tokens
        f.close()

        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [6]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('newdictionary')

2018-01-12 22:08:19,186 : INFO : loading Dictionary object from newdictionary
2018-01-12 22:08:19,314 : INFO : loaded newdictionary


In [7]:
######################################## load BOW
corpus = corpora.MmCorpus('newcorpus.mm')

2018-01-12 22:08:19,852 : INFO : loaded corpus index from newcorpus.mm.index
2018-01-12 22:08:19,853 : INFO : initializing corpus reader from newcorpus.mm
2018-01-12 22:08:20,056 : INFO : accepted corpus with 351985 documents, 35955 features, 88778565 non-zero entries


In [8]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [ ]:
# HAS BEEN SAVED
# capital C denotes collection
wc_C = {}
total_C = 0
for doc in corpus:
    for word,freq in doc:
        total_C += freq
        if word in wc_C:
            wc_C[word] += freq
        else:
            wc_C[word] = freq

            
# probability of a word in the colllection
p_C = {}            
for key in wc_C.iterkeys():
    p_C[key] = float(wc_C[key])/float(total_C)
    

with open('p_C.pkl', 'wb') as output:
    pickle.dump(p_C, output, pickle.HIGHEST_PROTOCOL)



In [9]:
# load p_C
with open('p_C.pkl', 'rb') as ip:
    p_C = pickle.load(ip)

In [ ]:
# list of document lengths in the corpus
doc_len = []
for doc in corpus:
    temp = 0
    for word, freq in doc:
        temp += freq 
    doc_len.append(temp)

with open('doc_len.pkl', 'wb') as output:
    pickle.dump(doc_len, output, pickle.HIGHEST_PROTOCOL)


In [10]:
# load doc_len
with open('doc_len.pkl', 'rb') as ip:
    doc_len = pickle.load(ip)

In [11]:

import math
from six import iteritems
from six.moves import xrange


# JM parameter
LAMBDA = 0.7


class JM(object):

    def __init__(self, corpus):
        self.corpus = corpus
        self.corpus_size = dictionary.num_docs

    
    def get_score(self, query, index, query_len):
        # word -> freq of doc.
        index_doc = dict(self.corpus[index])
        score = 0
        for word,freq in query:
            if word not in index_doc:
                continue
            p_ml = float(index_doc[word])/float(doc_len[index])
            
            score += freq*(math.log(1 + (1 - LAMBDA) * p_ml / (LAMBDA * p_C[word]) ))
        return score + query_len * math.log(LAMBDA)

    def get_scores(self, query):
        scores = []
        query_len = 0
        for word, freq in query:
            query_len += freq
        for index in xrange(self.corpus_size):
            score = self.get_score(query, index, query_len)
            scores.append(score)
        return scores



In [12]:
# run it once for initializing the idf 
jm = JM(corpus)

In [13]:
LAMBDA = 0.7

In [14]:
results = copy.deepcopy(query_truth)

sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    sims = jm.get_scores(dictionary.doc2bow(query))
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
    print i

    
        

0
1
2
3
4
5
6
7
8
9


In [15]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 16.90040454082415, 891],
  ['ConsumerCourt_DCDRC_106530', '0', 29.073265759456337, 243],
  ['ConsumerCourt_DCDRC_107608', '1', 8.02338809998778, 2418],
  ['ConsumerCourt_DCDRC_114291', '1', 33.07092922412956, 130],
  ['ConsumerCourt_DCDRC_114382', '1', 19.91059098496005, 637],
  ['ConsumerCourt_DCDRC_118185', '1', 12.915524936204736, 1359],
  ['ConsumerCourt_DCDRC_130318', '0', 38.45895884113922, 59],
  ['ConsumerCourt_DCDRC_130570', '1', 30.829072842643054, 197],
  ['ConsumerCourt_DCDRC_131146', '1', 17.979204275138837, 759],
  ['ConsumerCourt_DCDRC_131717', '1', 18.544149268054905, 720],
  ['ConsumerCourt_DCDRC_131741', '1', 32.457590006412346, 149],
  ['ConsumerCourt_DCDRC_131818', '1', 31.75766365068534, 168],
  ['ConsumerCourt_DCDRC_131950', '0', 20.729661799633117, 589],
  ['ConsumerCourt_DCDRC_131972', '0', 43.84604444931051, 21],
  ['ConsumerCourt_DCDRC_132932', '0', 28.651593064921745, 252],
  ['ConsumerCourt_DCDRC_133592', '1', 33.27

In [16]:
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
from scipy.stats import hmean


In [26]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'LM_JM.xlsx'
ws1 = wb.active
ws1.title = "Lambda=0.7|"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

In [27]:
#Query wise

import pandas as pd

f = pd.ExcelFile('LM_JM.xlsx')

from scipy.stats import hmean



model_name = 'LM_JM'
places = 4
wb = load_workbook('LM_JM.xlsx')
ws1 = wb.create_sheet(title="evaluation_Qwise")


ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
        
wb.save(filename = 'LM_JM.xlsx')

In [28]:
#Query wise Average precision

# function to calculate the precision sum(numerator in AP formula)
def p_sum(z):
    z = z.copy()
    z.sort_values(inplace=True)
    result = 0
    # enumerate - i gives the no. of relevant till now
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
        #print result
    return result


import pandas as pd

f = pd.ExcelFile('LM_JM.xlsx')

from scipy.stats import hmean


model_name = 'LM_JM'
places = 4
wb = load_workbook('LM_JM.xlsx')
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation" or sheet == "evaluation_Qwise"):
        continue
    print sheet
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name + ' :' + sheet]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name + ' :' + sheet]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);

        
wb.save(filename = 'LM_JM.xlsx')

Lambda=0.7|
